In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import gmaps
import requests as rq
from pprint import pprint
# Import API key
import config

# Configure gmaps
gkey = config.gkey
gmaps.configure(api_key=gkey)

In [2]:
# Create dataframe to store city data from csv
cities_df = pd.read_csv('Resources//weather_data.csv')
cities_df.dropna(inplace = True) 
cities_df.head()

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City DateTime
0,Torbay,47.67,-52.73,44.01,74,75,16.11,CA,2020-11-13 10:41:31
1,Hermanus,-34.42,19.23,66.00,79,91,5.01,ZA,2020-11-13 10:40:52
2,Yellowknife,62.46,-114.35,8.60,92,75,8.05,CA,2020-11-13 10:42:21
3,Cape Town,-33.93,18.42,75.00,68,0,17.22,ZA,2020-11-13 10:41:38
4,Grindavik,63.84,-22.43,37.40,86,75,17.22,IS,2020-11-13 10:41:49


In [3]:
# Store latitude and longitude in locations
locations = cities_df[["City Latitude", "City Longitude"]]
humidity = cities_df["Humidity"]

In [4]:
# Used for a point on the map to zoom into and format layout
zero_coor = (33.93,18.42)
figure_layout = {'width': '1050px', 'margin': '0 auto 0 auto'}
# Plot Heatmap
fig = gmaps.figure(center=zero_coor,zoom_level=2,layout=figure_layout,map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,max_intensity=.95)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='1050px'))

In [5]:
# Filter down dataframe based on unique selection
filtered_temp_df = cities_df.loc[(cities_df['Max Temperature'] < 90) & (cities_df['Max Temperature'] >= 65) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloud Coverage'] == 0)]
filtered_temp_df.describe()

,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed
count,31.000000,31.000000,31.000000,31.000000,31.0,31.000000
mean,6.306452,30.718065,75.080968,59.322581,0.0,5.754194
std,23.168493,80.751593,6.294859,20.695228,0.0,2.169582
min,-34.040000,-151.750000,65.370000,13.000000,0.0,1.010000
25%,-21.245000,4.695000,68.990000,47.000000,0.0,4.035000
50%,17.080000,44.400000,75.990000,60.000000,0.0,5.820000
75%,24.045000,71.125000,78.970000,78.000000,0.0,7.360000
max,34.130000,148.170000,86.200000,94.000000,0.0,9.190000


In [6]:
# Create two empty columns to store Hotel Data gathered from API Call
filtered_temp_df['Hotel Name'] = '' 
filtered_temp_df['Hotel Rating'] = '' 
filtered_temp_df.head()

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City DateTime,Hotel Name,Hotel Rating
26,Guerrero Negro,27.98,-114.06,68.18,59,0,8.59,MX,2020-11-13 10:43:14,,
63,Saint-Philippe,-21.36,55.77,77.00,78,0,3.36,RE,2020-11-13 10:41:48,,
69,Porbandar,21.64,69.61,80.08,43,0,9.19,IN,2020-11-13 10:44:48,,
89,Moranbah,-22.00,148.05,71.53,79,0,7.90,AU,2020-11-13 10:45:34,,
111,Najrān,17.49,44.13,75.20,29,0,3.60,SA,2020-11-13 10:46:24,,


In [7]:
# Store all cities from dataframe
cities = filtered_temp_df['City']

# Loop through the city list for Latitude and Longitude values and build parameters to pass to API call
for city in cities: 
    lat = filtered_temp_df.loc[filtered_temp_df['City'] == city, 'City Latitude'].values[0]
    lon = filtered_temp_df.loc[filtered_temp_df['City'] == city, 'City Longitude'].values[0]
    criteria = "Hotel"
    coords = f"{lat},{lon}"
    radius = 5000
    params = {
    "location": coords,
    "keyword": criteria,
    "radius": radius,
    "key": gkey
    }
    
    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run request
    hotel_info = rq.get(base_url, params).json()

    try:
        filtered_temp_df.loc[filtered_temp_df['City'] == city, 'Hotel Name'] = hotel_info["results"][0]['name']
        filtered_temp_df.loc[filtered_temp_df['City'] == city, 'Hotel Rating'] = hotel_info["results"][0]['rating']
    except:
        print(f'No hotel nearby {city}... skipping.')
    #pprint(hotel_info["results"])

No hotel nearby Najrān... skipping.
No hotel nearby Araouane... skipping.
No hotel nearby Ocós... skipping.
No hotel nearby Bilma... skipping.
No hotel nearby Dicabisagan... skipping.


In [8]:
# Select the first 10 rows for map display
filtered_temp_df = filtered_temp_df.head(10)
filtered_temp_df

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City DateTime,Hotel Name,Hotel Rating
26,Guerrero Negro,27.98,-114.06,68.18,59,0,8.59,MX,2020-11-13 10:43:14,The Halfway Inn,4.1
63,Saint-Philippe,-21.36,55.77,77.00,78,0,3.36,RE,2020-11-13 10:41:48,Les Embruns Du Baril,3.6
69,Porbandar,21.64,69.61,80.08,43,0,9.19,IN,2020-11-13 10:44:48,Lords Inn Porbandar,4.2
89,Moranbah,-22.00,148.05,71.53,79,0,7.90,AU,2020-11-13 10:45:34,Direct Hotels - Monterey Moranbah,4.8
111,Najrān,17.49,44.13,75.20,29,0,3.60,SA,2020-11-13 10:46:24,,
117,Araouane,18.90,-3.53,86.20,13,0,5.88,ML,2020-11-13 10:46:37,,
133,Faanui,-16.48,-151.75,79.21,75,0,8.21,PF,2020-11-13 10:41:42,InterContinental Bora Bora Resort Thalasso Spa,4.7
149,Yulara,-25.24,130.99,73.40,20,0,5.82,AU,2020-11-13 10:47:48,Ayers Rock Resort,4
184,Saint-Joseph,-21.37,55.62,77.00,78,0,3.36,RE,2020-11-13 10:49:07,Terre Rouge,5
189,Oudtshoorn,-33.59,22.20,71.60,49,0,6.93,ZA,2020-11-13 10:49:18,Protea Hotel by Marriott Oudtshoorn Riempie Es...,4.4


In [9]:
# Store coordinates and humidity from dataframe
locations = filtered_temp_df[["City Latitude", "City Longitude"]]
humidity = filtered_temp_df["Humidity"]

# Create a Description List Element to store information for clickable pin info
info_box_template = """
<dl>
<dt>City</dt>
<dd>{City}</dd>

<dt>Country</dt>
<dd>{City Country}</dd>

<dt>Hotel Name</dt>
<dd>{Hotel Name}</dd>

<dt>Rating</dt>
<dd>{Hotel Rating}</dd>
</dl>
"""

# Added feature for data for Hotel rating hoverable
rating_template = """Hotel Rating: {Hotel Rating}"""

# Plot Heatmap
fig = gmaps.figure(center=zero_coor,zoom_level=2,layout=figure_layout,map_type="TERRAIN")

marker = filtered_temp_df[['City Latitude','City Longitude']] 

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

info = [info_box_template.format(**row) for index, row in filtered_temp_df.iterrows()]
rating = [rating_template.format(**row) for index, row in filtered_temp_df.iterrows()]
# Add layer
markers = gmaps.marker_layer(marker,info_box_content=info, hover_text=rating)
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='1050px'))

In [10]:
#%reset